<a href="https://colab.research.google.com/github/Renita1206/Getting-Started-with-NLP/blob/main/IMDB_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [ ]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.5.0


In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

# Working with the data

In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
training_padded=np.array(padded)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)
testing_padded = np.array(testing_padded)


In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


# Training the Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(), # or GlobalAveragePooling1D layer
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
flatten_18 (Flatten)         (None, 1920)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 6)                 11526     
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(training_padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4855 - accuracy: 0.7518 - val_loss: 0.3542 - val_accuracy: 0.8447
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2333 - accuracy: 0.9097 - val_loss: 0.3664 - val_accuracy: 0.8416
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0832 - accuracy: 0.9798 - val_loss: 0.4448 - val_accuracy: 0.8298
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0215 - accuracy: 0.9970 - val_loss: 0.5328 - val_accuracy: 0.8260
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0055 - accuracy: 0.9997 - val_loss: 0.5894 - val_accuracy: 0.8280
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6431 - val_accuracy: 0.8286
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 8.6657e-04 - accuracy: 1.0000 - val_loss: 0.6772 - val_accuracy: 0.8304
Ep

# Visualizing the Data

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

https://projector.tensorflow.org/  
Link to visualize the vector space

# Output

In [ ]:
inp=input()
sentence = [inp]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")
pred=model.predict(padded)[0][0]
print(pred)
if(pred>0.5):
  print("The review is positive")
else:
  print("The review is negative")

Beautiful movie with a mind blowing story
0.96454054
The review is positive
